<a href="https://colab.research.google.com/github/auqa5325/AUQA-colab/blob/main/AUQA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install boto3 -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 93.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 5.8 MB/s eta 0:00:00


In [2]:
import os
from google.colab import userdata

AWS_ACCESS_KEY_ID     = userdata.get("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = userdata.get("AWS_SECRET_ACCESS_KEY")
#AWS_SESSION_TOKEN     = userdata.get("AWS_SESSION_TOKEN")  # may be None
AWS_REGION = userdata.get("AWS_REGION")

os.environ["AWS_ACCESS_KEY_ID"] = AWS_ACCESS_KEY_ID
os.environ["AWS_SECRET_ACCESS_KEY"] = AWS_SECRET_ACCESS_KEY
os.environ["AWS_REGION"] = AWS_REGION

print("✅ Credentials set. Region:", AWS_REGION)


✅ Credentials set. Region: ap-south-1


In [3]:
import boto3, os

REGION = os.environ["AWS_REGION"]
session = boto3.Session(region_name=REGION)
bedrock         = session.client("bedrock")
bedrock_runtime = session.client("bedrock-runtime")
print("✅ boto3 session initialized in:", session.region_name)


✅ boto3 session initialized in: ap-south-1


# output dict structure :
* id → unique identifier for this response (good for
logging/debugging).

* type → "message" → tells you this is a message object.

* role → "assistant" → the speaker role (assistant vs. user).

* model → which model gave this reply (claude-3-sonnet-20240229).

* content → a list of parts that make up the response.

* stop_reason → why the model stopped (e.g., end_turn, max_tokens).

* stop_sequence → custom sequence that stopped generation (here it’s None).

* usage → token usage info (handy for cost + rate limits).

In [4]:
import json
model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
prompt = "Hello, can you briefly introduce yourself?"

body = {
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 100,
    "messages": [
        {"role": "user", "content": [{"type": "text", "text": prompt}]}
    ]
}

resp = bedrock_runtime.invoke_model(modelId=model_id, body=json.dumps(body))
output = json.loads(resp["body"].read())
reply = output["content"][0]["text"]
usage = output.get("usage", {})
input_tokens = usage.get("input_tokens")
output_tokens = usage.get("output_tokens")
print("Prompt:\n",prompt)
print("Reply:\n", reply)
print("\n📊 Usage:")
print("  Input tokens :", input_tokens)
print("  Output tokens:", output_tokens)

ClientError: An error occurred (UnrecognizedClientException) when calling the InvokeModel operation: The security token included in the request is invalid.

In [ ]:
import json
model_id = "mistral.mixtral-8x7b-instruct-v0:1"
prompt = "explain tcp ip?"

body = {
    "prompt": prompt,
    "max_tokens": 100,
}

resp = bedrock_runtime.invoke_model(modelId=model_id, body=json.dumps(body))
output = json.loads(resp["body"].read())
print(output["outputs"][0]["text"])


{'outputs': [{'text': '\n\nTCP/IP is a suite of protocols that defines the Internet. Originally designed for the UNIX operating system. The TCP/IP protocol suite is a four-layer model that provides end-to-end connectivity specifying how data should be packetized, addressed, transmitted, routed, and received.\n\nThe four layers of the TCP/IP model are:\n\n1. The Application Layer: This is the topmost layer of the TCP/IP', 'stop_reason': 'length'}]}

In [ ]:
!pip -q install opensearch-py requests-aws4auth


import os, boto3, urllib.parse
from opensearchpy import OpenSearch, RequestsHttpConnection
from requests_aws4auth import AWS4Auth

OPENSEARCH_URL = "https://search-test1-annauniv-pcx3f52wxykhpd4md6v4bjeqdy.ap-south-1.es.amazonaws.com"
INDEX_NAME     = "test-annauniv"

REGION = os.environ["AWS_REGION"]  # you already set this above
host = urllib.parse.urlparse(OPENSEARCH_URL).netloc

session = boto3.Session(region_name=REGION)
creds   = session.get_credentials().get_frozen_credentials()
awsauth = AWS4Auth(creds.access_key, creds.secret_key, REGION, "es", session_token=creds.token)

os_client = OpenSearch(
    hosts=[{"host": host, "port": 443}],
    http_auth=awsauth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    timeout=60,                 # per-request socket timeout (default ~10s)
    max_retries=3,
    retry_on_timeout=True,
)

print("Ping:", os_client.ping())


In [ ]:
# Get index settings + mappings
info = os_client.indices.get(index=INDEX_NAME)
print(info)



In [ ]:
# Fetch all documents (up to 10 by default)
res = os_client.search(index=INDEX_NAME, body={"query": {"match_all": {}}})
for hit in res["hits"]["hits"]:
    print(hit["_id"], hit["_source"])


In [ ]:
import json
from typing import List

# Titan Text Embeddings v2 (1024 dims)
EMBED_MODEL_ID = "amazon.titan-embed-text-v2:0"

def embed_text(text: str) -> List[float]:
    payload = {
        "inputText": text,
        # "dimensions": 1024,  # optional for v2; uncomment if you want to be explicit
    }
    resp = bedrock_runtime.invoke_model(
        modelId=EMBED_MODEL_ID,
        body=json.dumps(payload),
        accept="application/json",
        contentType="application/json",

    )
    body = json.loads(resp["body"].read())
    # Titan v2 returns {"embedding": [floats...]}
    vec = body["embedding"]
    if len(vec) != 1024:
        raise ValueError(f"Expected 1024-dim vector, got {len(vec)}")
    return vec


In [ ]:
DOC_ID = "q-001"
DOC_TEXT = "Explain convolutional neural networks used in image recognition."

vec = embed_text(DOC_TEXT)

doc = {
    "text": DOC_TEXT,
    "vector_field": vec,
    # optional extras to fit your mapping:
    "course_id": "CS8791",
    "page_no": 1,
    "metadata": {"source": "notes.md", "page": 1},
}

resp = os_client.index(index=INDEX_NAME, id=DOC_ID, body=doc, refresh=True)
print("Indexed:", resp["result"])


In [ ]:
QUERY_TEXT = "Explain CNNs in image recognition"
qvec = embed_text(QUERY_TEXT)

hybrid_query = {
    "size": 5,
    "query": {
        "bool": {
            "should": [
                {   # semantic part
                    "knn": {
                        "vector_field": {
                            "vector": qvec,
                            "k": 5,
                           # "num_candidates": 100
                        }
                    }
                },
                {   # keyword part
                    "match": {
                        "text": QUERY_TEXT
                    }
                }
            ]
        }
    }
}

res = os_client.search(index=INDEX_NAME, body=hybrid_query)
for h in res["hits"]["hits"]:
    print(round(h["_score"],3), "→", h["_source"].get("text","")[:80])


In [ ]:
schema =  {
  "settings": {
    "number_of_shards": 5,
    "number_of_replicas": 1,
    "index": {
      "knn": True,
      "knn.algo_param.ef_search": 512
    }
  },
  "mappings": {
    "properties": {
      "course_id": {
        "type": "text",
        "fields": { "keyword": { "type": "keyword", "ignore_above": 256 } }
      },
      "text": {
        "type": "text",
        "fields": { "keyword": { "type": "keyword", "ignore_above": 256 } }
      },
      "page_no": { "type": "long" },
      "metadata": {
        "properties": {
          "source": {
            "type": "text",
            "fields": { "keyword": { "type": "keyword", "ignore_above": 256 } }
          },
          "page": { "type": "long" }
        }
      },
      "embedding": { "type": "float" },
      "vector_field": {
        "type": "knn_vector",
        "dimension": 1024,
        "method": {
          "name": "hnsw",
          "engine": "nmslib",
          "space_type": "cosinesimil",
          "parameters": { "ef_construction": 512, "m": 16 }
        }
      }
    }
  }
}
  # paste the JSON above as a Python dict


os_client.indices.create(index=INDEX_NAME, body=schema)
print("Created:", INDEX_NAME)



In [ ]:
import json, numpy as np

EMBED_MODEL_ID = "amazon.titan-embed-text-v2:0"

def embed_text(text: str):
    payload = {"inputText": text}
    resp = bedrock_runtime.invoke_model(
        modelId=EMBED_MODEL_ID,
        body=json.dumps(payload),
        accept="application/json",
        contentType="application/json",
    )
    body = json.loads(resp["body"].read())
    vec = body["embedding"]
    if len(vec) != 1024:
        raise ValueError(f"Expected 1024 dims, got {len(vec)}")
    return vec

def l2_normalize(vec):
    v = np.array(vec, dtype="float32")
    n = np.linalg.norm(v)
    return (v if n == 0 else v / n).tolist()

def embed_text_norm(text: str):
    return l2_normalize(embed_text(text))


In [ ]:
!pip install pypdf
from google.colab import files
from pypdf import PdfReader

def read_txt(path: str):
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        return f.read()

def read_pdf_pages(path: str):
    reader = PdfReader(path)
    pages = []
    for i, p in enumerate(reader.pages, start=1):
        try:
            t = p.extract_text() or ""
        except Exception:
            t = ""
        pages.append((i, t))
    return pages

print("Select files (.pdf or .txt)")
uploaded = files.upload()  # UI prompt
print("Uploaded:", list(uploaded.keys()))


In [ ]:
import re
from typing import List, Tuple

def clean_text(s: str) -> str:
    s = s.replace("\x00", " ")
    s = re.sub(r"[ \t]+", " ", s)
    s = re.sub(r"\n{3,}", "\n\n", s)
    return s.strip()

def chunk_text(text: str, chunk_size=900, overlap=150) -> List[str]:
    text = clean_text(text)
    if not text:
        return []
    chunks = []
    start = 0
    n = len(text)
    while start < n:
        end = min(n, start + chunk_size)
        # try to cut at a sentence boundary near the end
        window = text[start:end]
        cut = window.rfind(". ")
        if cut == -1 or cut < int(0.6 * len(window)):
            cut = len(window)
        chunk = window[:cut].strip()
        if chunk:
            chunks.append(chunk)
        start = start + max(1, cut - overlap)
    return chunks

def chunk_pdf_pages(pages: List[Tuple[int, str]], chunk_size=900, overlap=150):
    out = []
    for page_no, text in pages:
        for ch in chunk_text(text, chunk_size, overlap):
            out.append((page_no, ch))
    return out


In [ ]:
from uuid import uuid4
from opensearchpy.helpers import bulk

def bulk_index_chunks(index_name: str, chunks: List[Tuple[str, str, int, str]]):
    """
    chunks: list of tuples (text, course_id, page_no, source)
    """
    def gen():
        for text, course_id, page_no, source in chunks:
            if not text.strip():
                continue
            vec = embed_text_norm(text)
            yield {
                "_op_type": "index",
                "_index": index_name,
                "_id": str(uuid4()),
                "_source": {
                    "text": text,
                    "course_id": course_id,
                    "page_no": page_no,
                    "metadata": {"source": source, "page": page_no},
                    "vector_field": vec
                }
            }
    ok, fail = bulk(os_client, gen(), chunk_size=200, request_timeout=120)
    os_client.indices.refresh(index=index_name)
    return ok, fail


In [ ]:
# Configure this per your dataset
DEFAULT_COURSE_ID = "DSA"

ingest_items = []  # (text, course_id, page_no, source)

for fname in uploaded.keys():
    path = f"/content/{fname}"
    if fname.lower().endswith(".pdf"):
        pages = read_pdf_pages(path)
        for page_no, txt in chunk_pdf_pages(pages):
            ingest_items.append((txt, DEFAULT_COURSE_ID, page_no, fname))
    elif fname.lower().endswith(".txt"):
        txt = read_txt(path)
        for i, ch in enumerate(chunk_text(txt), start=1):
            ingest_items.append((ch, DEFAULT_COURSE_ID, i, fname))
    else:
        print("Skipping unsupported:", fname)

print("Chunks prepared:", len(ingest_items))
#ok, fail = bulk_index_chunks(INDEX_NAME, ingest_items)
#print(f"Indexed ok={ok}, failed={fail}")


In [ ]:
import hashlib
import time, random, concurrent.futures as cf

def make_id(source: str, page_no: int, text: str) -> str:
    raw = f"{source}-{page_no}-{text}".encode("utf-8")
    return hashlib.sha1(raw).hexdigest()
def enable_bulk_mode(index):
    os_client.indices.put_settings(index=index, body={"index": {
        "refresh_interval": "-1", "number_of_replicas": 0
    }})

def disable_bulk_mode(index, replicas=1):
    os_client.indices.put_settings(index=index, body={"index": {
        "refresh_interval": "1s", "number_of_replicas": replicas
    }})
    os_client.indices.refresh(index=index)


def _with_retry(fn, max_tries=5, base=0.5, cap=8.0):
    delay = base
    for i in range(max_tries):
        try:
            return fn()
        except Exception:
            if i == max_tries - 1: raise
            time.sleep(delay + random.random()*0.2)
            delay = min(delay*2, cap)

def embed_many(texts, max_workers=8):
    def one(t): return _with_retry(lambda: embed_text_norm(t))
    with cf.ThreadPoolExecutor(max_workers=max_workers) as ex:
        return list(ex.map(one, texts))


In [ ]:
from uuid import uuid4
from opensearchpy.helpers import bulk
import time

def stream_bulk(index_name, items_iter, batch_size=64, os_chunk_size=1000, workers=8):
    """
    items_iter yields tuples: (text, course_id, page_no, source)
    """
    total_ok = total_fail = 0
    seen = 0
    t0 = time.time()

    enable_bulk_mode(index_name)
    try:
        batch = []
        for itm in items_iter:
            batch.append(itm)
            if len(batch) >= batch_size:
                total_ok, total_fail, seen = _flush_batch(
                    index_name, batch, total_ok, total_fail, seen,
                    os_chunk_size=os_chunk_size, workers=workers, t0=t0
                )
                batch = []
        if batch:
            total_ok, total_fail, seen = _flush_batch(
                index_name, batch, total_ok, total_fail, seen,
                os_chunk_size=os_chunk_size, workers=workers, t0=t0
            )
    finally:
        disable_bulk_mode(index_name, replicas=1)

    elapsed = time.time() - t0
    print(f"\nDone. ok={total_ok}, failed={total_fail}, elapsed={elapsed:.1f}s, rate={seen/elapsed:.1f} docs/s")
    return total_ok, total_fail

def _flush_batch(index_name, batch, total_ok, total_fail, seen, *, os_chunk_size, workers, t0):
    texts  = [t for (t,_,_,_) in batch]
    vecs   = embed_many(texts, max_workers=workers)

    actions = []
    for (text, course_id, page_no, source), vec in zip(batch, vecs):
        if not text.strip():
            continue
        actions.append({
            "_op_type": "index",
            "_index": index_name,
            "_id": make_id(source, page_no, text),  # deterministic ID (if you added make_id)
            "_source": {
                "text": text,
                "course_id": course_id,
                "page_no": page_no,
                "metadata": {"source": source, "page": page_no},
                "vector_field": vec
            }
        })

    ok, fail = 0, 0
    if actions:
        # errors is a LIST; don't try to += it into an int
        ok, errors = bulk(
            os_client,
            actions,
            chunk_size=os_chunk_size,
            request_timeout=300,
            raise_on_error=False,      # collect errors instead of raising
        )
        fail = len(errors) if isinstance(errors, list) else int(errors or 0)
        if fail:
            # optional: peek at a couple of errors for visibility
            print("Sample errors:", errors[:2])

    seen += len(actions)
    total_ok   += ok
    total_fail += fail

    elapsed = time.time() - t0
    rate = (seen/elapsed) if elapsed > 0 else 0
    print(f"seen={seen} ok+={ok} fail+={fail} elapsed={elapsed:.1f}s rate≈{rate:.1f} docs/s")
    return total_ok, total_fail, seen

def iter_ingest_items(ingest_items):
    for it in ingest_items:
        yield it

# tune these 3 knobs to match your limits:
OK, FAIL = stream_bulk(
    INDEX_NAME,
    items_iter=iter_ingest_items(ingest_items),
    batch_size=96,
    os_chunk_size=500,     # see (B) below
    workers=12
)



In [ ]:
def hybrid_search(query_text: str, k=5,  course_id: str=None):
    qvec = embed_text_norm(query_text)

    must_clause = [
        {"knn": {"vector_field": {"vector": qvec, "k": k}}}
    ]
    should_clause = [
        {"match": {"text": {"query": query_text, "boost": 2.0}}}  # keyword gets a little boost
    ]

    body = {
        "size": k,
        "query": {
            "bool": {
                "must": must_clause,
                "should": should_clause,
                "minimum_should_match": 0,
                "filter": []
            }
        }
    }
    if course_id:
        body["query"]["bool"]["filter"].append({"term": {"course_id.keyword": course_id}})

    res = os_client.search(index=INDEX_NAME, body=body)
    return [
        (round(h["_score"], 3),
         h["_source"].get("text","")[:].replace("\n"," "),
         h["_source"].get("metadata",{}).get("source"),
         h["_source"].get("metadata",{}).get("page"),
         h["_id"])
        for h in res["hits"]["hits"]
    ]

# Try it:
results = hybrid_search("Heap Sort Algorithm", k=10, course_id=DEFAULT_COURSE_ID)
for r in results:
    print(r)
